In [4]:
# STEP 1: Install Required Packages
!pip install -q faiss-cpu sentence-transformers transformers

# STEP 2: Upload Dataset CSVs
from google.colab import files
uploaded = files.upload()


Saving Sample_Submission.csv to Sample_Submission (1).csv
Saving Test Dataset.csv to Test Dataset (1).csv
Saving Training Dataset.csv to Training Dataset (1).csv


In [5]:
# STEP 3: Read Uploaded CSV Files
import pandas as pd

train_df = pd.read_csv("Training Dataset.csv")
test_df = pd.read_csv("Test Dataset.csv")

print("Train shape:", train_df.shape)
train_df.head()


Train shape: (614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [6]:
# STEP 4: Convert Each Row into a Text Document
documents = []

for idx, row in train_df.iterrows():
    doc = f"""Applicant {idx}:
    Gender = {row['Gender']},
    Married = {row['Married']},
    Education = {row['Education']},
    Self_Employed = {row['Self_Employed']},
    ApplicantIncome = {row['ApplicantIncome']},
    CoapplicantIncome = {row['CoapplicantIncome']},
    LoanAmount = {row['LoanAmount']},
    Loan_Amount_Term = {row['Loan_Amount_Term']},
    Credit_History = {row['Credit_History']},
    Property_Area = {row['Property_Area']},
    Loan_Status = {row['Loan_Status']}
    """
    documents.append(doc.strip())


In [7]:
# STEP 5: Create Embeddings and Build FAISS Index
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedding_model.encode(documents, show_progress_bar=True)

dimension = doc_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(np.array(doc_embeddings))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/20 [00:00<?, ?it/s]

In [8]:
# STEP 6: Define Document Retrieval Function
def retrieve_docs(query, k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = faiss_index.search(np.array(query_embedding), k)
    results = [documents[i] for i in indices[0]]
    return results


In [9]:
# STEP 7: Use HuggingFace Model for Response Generation
from transformers import pipeline

qa_model = pipeline("text2text-generation", model="google/flan-t5-base")

def answer_query(query):
    context = "\n".join(retrieve_docs(query))
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    answer = qa_model(prompt, max_length=150)[0]['generated_text']
    return answer.strip()


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [11]:
# STEP 8: Test with a Single Query (NO LOOP)

# 👇 Change your question here
query = "Do people with low credit history get loans?"

# Get answer from the RAG chatbot
response = answer_query(query)

# Print the result
print("Question:", query)
print("Answer:", response)


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Question: Do people with low credit history get loans?
Answer: Yes
